In [ ]:
import sys
sys.path.append('../input/nfnets-keras')

!pip install --quiet ../input/keras-efficientnet-whl/Keras_Applications-1.0.8-py3-none-any.whl
# !pip install --quiet ../input/keras-efficientnet-whl/efficientnet-1.1.1-py3-none-any.whl

import os
import math
import re
import random
import tensorflow as tf
import tensorflow_addons as tfa
import numpy as np
import tensorflow.keras.backend as K
# import efficientnet.keras as efn
# import efficientnet
import itertools
import matplotlib
import scipy
import pandas as pd
import sklearn
from matplotlib import pyplot as plt
from datetime import datetime
from functools import partial
from kaggle_datasets import KaggleDatasets
import pickle
from sklearn.preprocessing import StandardScaler
from nfnet import NFNet, nfnet_params

# import cudf
# import cuml
# import cupy
# from cuml.feature_extraction.text import TfidfVectorizer
# from cuml.neighbors import NearestNeighbors

from sklearn.neighbors import NearestNeighbors
from sklearn.feature_extraction.text import TfidfVectorizer

SEED = 42

random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

In [ ]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Device: ', tpu.master())
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
    DEVICE = "TPU"
except:
    DEVICE = 'notTPU'
    strategy = tf.distribute.get_strategy()
print(DEVICE)

FOLD = 3
AUTO = tf.data.experimental.AUTOTUNE
GCS_PATH = KaggleDatasets().get_gcs_path(f'shopee-tfrecord-512-gkf-4-folds-230421')
REPLICAS = strategy.num_replicas_in_sync
BATCH_SIZE = 32 * REPLICAS
FILENAMES = tf.io.gfile.glob(GCS_PATH + f'/fold_{FOLD}/*.tfrec')
FILENAMES

In [ ]:
IMAGE_SIZE = (512, 512)

EPOCHS = 100
LR = 0.001

CLASSES = 11014

In [ ]:
def decode_image(image, img_size):
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, img_size)
    image = tf.cast(image, tf.float32) / 255.0
    return tf.reshape(image, [*img_size, 3])

# parse tfrecord files
def _parse_image(proto, train):
    features = {
        "image": tf.io.FixedLenFeature([], tf.string), # tf.string means bytestring
        "title_vec": tf.io.FixedLenFeature([100], tf.float32),  # shape [] means single element
        "phash_vec": tf.io.FixedLenFeature([25], tf.float32),
        "label": tf.io.FixedLenFeature([], tf.int64),  
    }
    return tf.io.parse_single_example(proto, features)

# get image and label_group
def get_image_and_label(proto, train, img_size):
    sample = _parse_image(proto, train=train)
    sample["image"] = decode_image(sample["image"], img_size)
    image = sample["image"]
    title_vec = sample["title_vec"]
    phash_vec = sample["phash_vec"]
    label = tf.cast(sample['label'], tf.int32)
#     label = tf.one_hot(tf.cast(sample['label'], tf.int32), len(CLASSES))
    return (image, title_vec, label), label

def get_training_dataset(tfr, batchsize, img_size = (512,512)):
    return tfr.map(partial(get_image_and_label, train=True, img_size=img_size)).repeat().map(
        lambda x,y: image_augmentation(x,y)).shuffle(1000).batch(batchsize).prefetch(AUTO)

def get_validation_dataset(tfr, batchsize, img_size=(512, 512), train=True):
    return tfr.map(partial(get_image_and_label, train=train, img_size=img_size)).batch(batchsize).prefetch(AUTO)

def get_tfrecord_size(tfrecord):
    return sum(1 for _ in tfrecord)

In [ ]:
# Augmentation function & tf.image functions (flip, brightness, contrast, saturation, hue)
# for train
# @tf.function
def image_augmentation(iw, label):
    image, title_vec, label = iw

    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)
    image = tf.image.random_hue(image, 0.01)
    image = tf.image.random_saturation(image, 0.70, 1.30)
    image = tf.image.random_contrast(image, 0.80, 1.20)
    image = tf.image.random_brightness(image, 0.10)

    return ((image, title_vec, label), label)

In [ ]:
from tensorflow.keras.models import Model, Sequential
import keras.backend as K
from keras.optimizers import SGD
from tensorflow.keras.layers import (
    Input, 
    Flatten, 
    Dense, 
    Dropout, 
    AveragePooling2D, 
    GlobalAveragePooling2D, 
    SpatialDropout2D, 
    BatchNormalization, 
    Activation, 
    Concatenate,
    Embedding,
    GlobalAveragePooling1D,
    Lambda
)

In [ ]:
def plt_lr(epoch_count):
    if epoch_count > 50:
        epoch_count = 50
    
    rng = [i for i in range(epoch_count)]

    plt.figure()
    y = [lrfn(x) for x in rng]
    plt.title(f'Learning rate schedule: {y[0]} to {y[epoch_count-1]}')
    plt.plot(rng, y)

def plt_acc(h):
    plt.figure()
    plt.plot(h.history["sparse_categorical_accuracy"])
    if 'val_sparse_categorical_accuracy' in h.history:
        plt.plot(h.history["val_sparse_categorical_accuracy"]) 
        plt.legend(["training","validation"])       
    else:
        plt.legend(["training"])
    plt.xlabel("epoch")
    plt.title("Sparse Categorical Accuracy")
    plt.show()
    
def plt_f1_score(h):
    plt.figure()
    plt.plot(h.history["f1_score"])
    if 'f1_score' in h.history:
        plt.plot(h.history["f1_score"]) 
        plt.legend(["training","validation"])       
    else:
        plt.legend(["training"])
    plt.xlabel("epoch")
    plt.title("F1 Score")
    plt.show()

def plt_loss(h):
    plt.figure()
    plt.plot(h.history["loss"])
    if 'val_loss' in h.history:
        plt.plot(h.history["val_loss"]) 
        plt.legend(["training","validation"])       
    else:
        plt.legend(["training"])
    plt.legend(["training","validation"])
    plt.xlabel("epoch")
    plt.title("Loss")
    plt.show()

In [ ]:
es_val_f1 = tf.keras.callbacks.EarlyStopping(
    monitor='f1_score', min_delta=0.001, patience=10, verbose=1, mode='max',
    baseline=None, restore_best_weights=True
)

es_val_f1_stage2 = tf.keras.callbacks.EarlyStopping(
    monitor='f1_score', min_delta=0.001, patience=15, verbose=1, mode='max',
    baseline=None, restore_best_weights=True
)

cb_checkpoint = tf.keras.callbacks.ModelCheckpoint(
    "backup_weights_nfnet.h5",
    monitor="f1_score", verbose=1, mode='max', save_best_only=True, save_weights_only=True)

cb_checkpoint._supports_tf_logs = False

In [ ]:
# swish activation
from keras.backend import sigmoid
def swish(x, beta = 1):
    return (x * sigmoid(beta * x))

from keras.utils.generic_utils import get_custom_objects
from keras.layers import Activation

get_custom_objects().update({'swish': Activation(swish)})

In [ ]:
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation
from keras.layers import Bidirectional, GlobalMaxPool1D,Bidirectional, Conv1D, GlobalMaxPooling1D, Conv2D
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
class ArcMarginProduct(keras.layers.Layer):
    '''
    Implements large margin arc distance.

    Reference:
        https://arxiv.org/pdf/1801.07698.pdf
        https://github.com/lyakaap/Landmark2019-1st-and-3rd-Place-Solution/
            blob/master/src/modeling/metric_learning.py
    '''
    def __init__(self, n_classes, s=30, m=0.50, easy_margin=False,
                 ls_eps=0.0, **kwargs):

        super(ArcMarginProduct, self).__init__(**kwargs)

        self.n_classes = n_classes
        self.s = s
        self.m = m
        self.ls_eps = ls_eps
        self.easy_margin = easy_margin
        self.cos_m = tf.math.cos(m)
        self.sin_m = tf.math.sin(m)
        self.th = tf.math.cos(math.pi - m)
        self.mm = tf.math.sin(math.pi - m) * m

    def get_config(self):

        config = super().get_config().copy()
        config.update({
            'n_classes': self.n_classes,
            's': self.s,
            'm': self.m,
            'ls_eps': self.ls_eps,
            'easy_margin': self.easy_margin,
        })
        return config

    def build(self, input_shape):
        super(ArcMarginProduct, self).build(input_shape[0])

        self.W = self.add_weight(
            name='W',
            shape=(int(input_shape[0][-1]), self.n_classes),
            initializer='glorot_uniform',
            dtype='float32',
            trainable=True,
            regularizer=None)

    def call(self, inputs):
        X, y = inputs
        y = tf.cast(y, dtype=tf.int32)
        cosine = tf.matmul(
            tf.math.l2_normalize(X, axis=1),
            tf.math.l2_normalize(self.W, axis=0)
        )
        sine = tf.math.sqrt(1.0 - tf.math.pow(cosine, 2))
        phi = cosine * self.cos_m - sine * self.sin_m
        if self.easy_margin:
            phi = tf.where(cosine > 0, phi, cosine)
        else:
            phi = tf.where(cosine > self.th, phi, cosine - self.mm)
        one_hot = tf.cast(
            tf.one_hot(y, depth=self.n_classes),
            dtype=cosine.dtype
        )
        if self.ls_eps > 0:
            one_hot = (1 - self.ls_eps) * one_hot + self.ls_eps / self.n_classes

        output = (one_hot * phi) + ((1.0 - one_hot) * cosine)
        output *= self.s
        return output

In [ ]:
GCS_PATH = KaggleDatasets().get_gcs_path(f'nfnet-model-checkpoints-270421')
MODEL_FILENAMES = tf.io.gfile.glob(GCS_PATH + f'/NFNET-F0/*')

def get_nfnet(weights='gs://kds-b135c9e4ed7b978eb1b05abe18f78523f1fe076527cd1f64555bd7b1/NFNET-F0/F0_NFNet'):
    nfnet_ = NFNet(
        num_classes=1000,
        variant='F0',
        drop_rate=0.2,
        label_smoothing=0.1,
        ema_decay=0.99999,
        clipping_factor=0.01,
        include_top=False,
    )
    if weights is not None:
        nfnet_.load_weights(weights)

    model_image = Sequential([
        nfnet_,
        GlobalAveragePooling2D(name='effb1-pooling'),
        BatchNormalization(name='effb1_bn1'),
        Dropout(0.2),
        Dense(2400, name='effb1_dense1'),
        Activation('swish', name='effb1_act1'),
    ], name='effb1-img')
    model_image.build((None, *IMAGE_SIZE, 3))
    
    model_title = Sequential([
        Input(shape=(100,), name='title-input'),
        Embedding(25000, 150, input_length=100, name='title-embed'),
        Dropout(0.2),
        Conv1D(300, 3, padding='valid', activation='relu', strides=1, name='title-conv'),
        GlobalMaxPool1D(name='title-globalMax'),
        Dense(720, name='title-dense1'), #650 -> 0.81
        Activation('swish', name='title-act1'),
        Dropout(0.2),
        
        Dense(650, name='title-dense2'),
        BatchNormalization(name='title-bn2'),
        Activation('swish', name='title-act2'),
    ], name='title-vec')
    
    margin = ArcMarginProduct(
        n_classes = CLASSES, 
        s = 30, 
        m = 0.7, 
        name='head/arc_margin', 
        dtype='float32'
    )

    concatenate = Concatenate(name='concatenate')([model_image.output, model_title.output])
    label = Input(shape=(), name='arc-input')
    arc_face = margin([concatenate, label])
    output = Dense(CLASSES, activation='softmax', name='output')(arc_face)
    
    model = Model(inputs=[model_image.input, model_title.input, label], outputs=output)
    return model

# model = get_nfnet()
# model.summary()

In [ ]:
def get_lr_callback():
    lr_start   = 0.000001
    lr_max     = 0.000005 * BATCH_SIZE
    lr_min     = 0.000001
    lr_ramp_ep = 5
    lr_sus_ep  = 0
    lr_decay   = 0.8
   
    def lrfn(epoch):
        if epoch < lr_ramp_ep:
            lr = (lr_max - lr_start) / lr_ramp_ep * epoch + lr_start   
        elif epoch < lr_ramp_ep + lr_sus_ep:
            lr = lr_max    
        else:
            lr = (lr_max - lr_min) * lr_decay**(epoch - lr_ramp_ep - lr_sus_ep) + lr_min    
        return lr

    lr_callback = tf.keras.callbacks.LearningRateScheduler(lrfn, verbose = True)
    return lr_callback

In [ ]:
tf.tpu.experimental.initialize_tpu_system(tpu)

# custom callback for f1 evaluation metric
# val_data = pd.read_parquet(f'../input/shopee-tfrecord-512-gkf-4-folds-230421/fold_{FOLD}/unseen.parquet', engine='pyarrow')
# tmp = val_data.groupby('label_group').posting_id.agg('unique').to_dict()
# val_data['target'] = val_data.label_group.map(tmp)
# del tmp
# customF1Score = CustomF1Score(validation_data = val_data)

# for randomness
ignore_order = tf.data.Options()
ignore_order.experimental_deterministic = False

train_records = tf.data.TFRecordDataset(FILENAMES, num_parallel_reads=AUTO)
train_records = train_records.with_options(ignore_order)
train_data = get_training_dataset(train_records, BATCH_SIZE, IMAGE_SIZE)
train_data = train_data.map(lambda x, y: (x, y))    
train_size = get_tfrecord_size(train_records)

In [ ]:
# fold 0: 22
# fold 1: 24
# fold 2: 14
# fold 3: 12
# avg: (22+24+14+12)/4 = 18
# total epoch: 18 + 18*25/100 = 22.5 = 23

with strategy.scope():
    model = get_nfnet()
    model.compile(
        optimizer = tfa.optimizers.LAMB(LR),
        loss = [tf.keras.losses.SparseCategoricalCrossentropy()],
        metrics = [tf.keras.metrics.SparseCategoricalAccuracy()]
    )
model.summary()
    
model.fit(
    train_data, 
    steps_per_epoch=train_size//BATCH_SIZE,
    epochs = 23,
    callbacks=[get_lr_callback()], verbose=1
)

h = model.history
plt_acc(h)
plt_loss(h)

In [ ]:
model.save_weights('model_weights.h5')